## Understanding Backpropagation
        Problem Statement. We are given some function f(x) where x is a vector of inputs ans we are interested in computing the gradient of f at x

Link: https://cs231n.github.io/optimization-2/
        
        Motivation. Recall that the primary reason we are interested in this problem is that the specific case of neural networks, f will corespond to the
 loss function L and the inputs x will consist of the training data and the neural network weights.For example, the loss could be the SVM loss function and the inputs are both the training data (xi,yi),i=1…N and the weights and biases W,b. Note that (as is usually the case in Machine Learning) we think of the training data as given and fixed, and of the weights as variables we have control over. Hence, even though we can easily use backpropagation to compute the gradient on the input examples xi, in practice we usually only compute the gradient for the parameters (e.g. W,b) so that we can use it to perform a parameter update. However, as we will see later in the class the gradient on xi can still be useful sometimes, for example for purposes of visualization and interpreting what the Neural Network might be doing.

## Simple expressions and interpretation of the gradient

Interpretation. Keep in mind what the derivatives tell you: They indicate the rate of change of a function with respect to that variable surrounding an infinitesimally small region near a particular point

A technical note is that the division sign on the left-hand side is, unlike the division sign on the right-hand side, not a division. Instead, this notation indicates that the operator d/dx is being applied to the function f, and returns a different function (the derivative). A nice way to think about the expression above is that when h is very small, then the function is well-approximated by a straight line, and the derivative is its slope. In other words, the derivative on each variable tells you the sensitivity of the whole expression on its value. For example, if x=4,y=−3 then f(x,y)=−12 and the derivative on x ∂f/∂x=−3. This tells us that if we were to increase the value of this variable by a tiny amount, the effect on the whole expression would be to decrease it (due to the negative sign), and by three times that amount. This can be seen by rearranging the above equation ( f(x+h)=f(x)+h*df(x)/dx ). Analogously, since ∂f/∂y=4, we expect that increasing the value of y by some very small amount h would also increase the output of the function (due to the positive sign), and by 4h.

### The derivative on each variable tells you the sensitivity of the whole expression on its value.

As mentioned, the gradient ∇f is the vector of partial derivatives, so we have that ∇f=[∂f/∂x,∂f/∂y]=[y,x]. Even though the gradient is technically a vector, we will often use terms such as “the gradient on x” instead of the technically correct phrase “the partial derivative on x” for simplicity.

In [1]:
# set some inputs
x = -2
y = 5
z = -4

# perform forward pass
q = x + y
f = q * z

# perform the backward pass in reverse order
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4

# now backprop through f = q * z
dfdx = 1.0 * dfdq # dq/dx = 1. And the multiplication here is the chain rule!
dfdy = 1.0 * dfdq # dq/dy = 1

We are left with the gradient in the variables [dfdx,dfdy,dfdz], which tell us the sensitivity of the variables x,y,z on f!. This is the simplest example of backpropagation. Going forward, we will use a more concise notation that omits the df prefix. For example, we will simply write dq instead of dfdq, and always assume that the gradient is computed on the final output.

## Intuitive understanding of backpropagation

Notice that backpropagation is a beautifully local process. Every gate in a circuit diagram gets some inputs and can right away compute two things: 1. its output value and 2. the local gradient of its output with respect to its inputs. Notice that the gates can do this completely independently without being aware of any of the details of the full circuit that they are embedded in. However, once the forward pass is over, during backpropagation the gate will eventually learn about the gradient of its output value on the final output of the entire circuit. Chain rule says that the gate should take that gradient and multiply it into every gradient it normally computes for all of its inputs.

    This extra multiplication (for each input) due to the chain rule can turn a single and relatively useless gate into a cog in a complex circuit such as an entire neural network.

Lets get an intuition for how this works by referring again to the example. The add gate received inputs [-2, 5] and computed output 3. Since the gate is computing the addition operation, its local gradient for both of its inputs is +1. The rest of the circuit computed the final value, which is -12. During the backward pass in which the chain rule is applied recursively backwards through the circuit, the add gate (which is an input to the multiply gate) learns that the gradient for its output was -4. If we anthropomorphize the circuit as wanting to output a higher value (which can help with intuition), then we can think of the circuit as “wanting” the output of the add gate to be lower (due to negative sign), and with a force of 4. To continue the recurrence and to chain the gradient, the add gate takes that gradient and multiplies it to all of the local gradients for its inputs (making the gradient on both x and y 1 * -4 = -4). Notice that this has the desired effect: If x,y were to decrease (responding to their negative gradient) then the add gate’s output would decrease, which in turn makes the multiply gate’s output increase.

Backpropagation can thus be thought of as gates communicating to each other (through the gradient signal) whether they want their outputs to increase or decrease (and how strongly), so as to make the final output value higher.

## Modularity: Sigmoid Example

In [4]:
import math
w = [2, -3, -3]
x = [-1, -2]

#forward pass
dot = w[0] * x[0] + w[1] * x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function
print('f = ',f)

# backward pass
ddot = (1-f) * f                                # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot]                 # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot]     # backprop into w

print("ddot = ", ddot)
print("dx = ", dx)
print("dw = ", dw)


f =  0.7310585786300049
ddot =  0.19661193324148185
dx =  [0.3932238664829637, -0.5898357997244456]
dw =  [-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]


## Backpropagation in practice: Staged computation

In [6]:
x = 3; y = 4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden #                                       #(8)

print("f =", f)

f = 0.07971588771237584


In [ ]:
# backprop f = num * invden
dnum = invden # gradient on numerator - derivate of num
dinvden = num # derivative of invden

# backprop invden = 1.0 / den
dden = (-1.0 / (den**2)) * dinvden

# backprop den = sigx + xpysqr
dsigx = (1) * dden
dxpysqr = (1) * dden

# backprop xpysqr = xpy ** 2
dxpy = (2 * xpy) * dxpysqr

# backprop xpy = x + y
dx = 1 * dxpy
dy = 1 * dxpy

# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx

# backprop num = x + sigy
dx += (1) * dnum
dsigy = (1) * dnum

#backprop sigy = 1.0 / (1 + math.exp(-y))
dy = ((1 - sigy) * sigy) * dsigy


Notice a few things:

Cache forward pass variables. To compute the backward pass it is very helpful to have some of the variables that were used in the forward pass. In practice you want to structure your code so that you cache these variables, and so that they are available during backpropagation. If this is too difficult, it is possible (but wasteful) to recompute them.

Gradients add up at forks. The forward expression involves the variables x,y multiple times, so when we perform backpropagation we must be careful to use += instead of = to accumulate the gradient on these variables (otherwise we would overwrite it). This follows the multivariable chain rule in Calculus, which states that if a variable branches out to different parts of the circuit, then the gradients that flow back to it will add.

## Patterns in backward flow

<img src="patterns in backprop.png" style="width:800px;height:400px;">

## Gradients for vectorized operations

The above sections were concerned with single variables, but all concepts extend in a straight-forward manner to matrix and vector operations. However, one must pay closer attention to dimensions and transpose operations.

Matrix-Matrix multiply gradient. Possibly the most tricky operation is the matrix-matrix multiplication (which generalizes all matrix-vector and vector-vector) multiply operations:

In [16]:
import numpy as np 
#forward pass
W = np.random.randn(5,10)
X = np.random.randn(10,3)
D = np.dot(W,X)
print("D=", D)


# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) #same shape as D
dW = np.dot(dD,X.T) 
dX = np.dot(W.T, dD)




D= [[-2.72378525 -1.12630746 -1.02767859]
 [-1.30695786  3.26759636 -2.71746943]
 [-0.5380469   9.30712498 -1.27375268]
 [-1.99834114  4.54807073 -3.10320621]
 [-0.59111737 12.74994047 -1.12904952]]


Tip: use dimension analysis! Note that you do not need to remember the expressions for dW and dX because they are easy to re-derive based on dimensions. For instance, we know that the gradient on the weights dW must be of the same size as W after it is computed, and that it must depend on matrix multiplication of X and dD (as is the case when both X,W are single numbers and not matrices). There is always exactly one way of achieving this so that the dimensions work out. For example, X is of size [10 x 3] and dD of size [5 x 3], so if we want dW and W has shape [5 x 10], then the only way of achieving this is with dD.dot(X.T), as shown above.

Work with small, explicit examples. Some people may find it difficult at first to derive the gradient updates for some vectorized expressions. Our recommendation is to explicitly write out a minimal vectorized example, derive the gradient on paper and then generalize the pattern to its efficient, vectorized form.